In [2]:
import yfinance as yf
from datetime import datetime, timedelta

def calculate_normalized_volume_change(ticker, target_date):
    try:
        # Ensure target_date is in correct format
        target_date = datetime.strptime(target_date, "%Y-%m-%d").date()
        
        # Fetch data for the last 7 days + the target date
        start_date = target_date - timedelta(days=7)
        end_date = target_date + timedelta(days=1)  # Include the target date
        
        # Fetch historical data
        stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date)
        
        # Ensure data for the target date is available
        if target_date not in stock_data.index.date:
            raise ValueError(f"No data available for {ticker} on {target_date}")
        
        # Extract volume for the target date
        target_date_volume = stock_data.loc[stock_data.index.date == target_date, 'Volume'].iloc[0]
        
        # Calculate weekly average volume (excluding target date)
        weekly_data = stock_data.loc[stock_data.index.date != target_date, 'Volume']
        weekly_avg_volume = weekly_data.mean()
        
        # Calculate Normalized Relative Volume Change
        if weekly_avg_volume > 0:
            normalized_rvc = abs((target_date_volume - weekly_avg_volume) / weekly_avg_volume * 100)
        else:
            normalized_rvc = 0  # Avoid division by zero
        
        return normalized_rvc
    except Exception as e:
        return {"Error": str(e)}

# Example usage
ticker_date_pairs = [
    {"ticker": "AAPL", "date": "2025-01-22"},
    {"ticker": "MSFT", "date": "2025-01-22"},
    {"ticker": "TSLA", "date": "2025-01-22"}
]

# Calculate and output Normalized Volume Change for each
for item in ticker_date_pairs:
    result = calculate_normalized_volume_change(item['ticker'], item['date'])
    if isinstance(result, dict) and "Error" in result:
        print(f"Error for {item['ticker']}: {result['Error']}")
    else:
        print(f"{item['ticker']}: Normalized Volume Change: {result:.2f}")


AAPL: Normalized Volume Change: 7.78
MSFT: Normalized Volume Change: 27.51
TSLA: Normalized Volume Change: 26.56
